In [1]:
! pip install --quiet numpy pandas seaborn matplotlib tqdm
! pip install --quiet datasets "transformers[torch]" scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Rice Leaf Disease Detection - Model Evaluation
This notebook evaluates multiple transformer-based models for rice leaf disease classification.

In [ ]:

import os
import json
import torch
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoModelForImageClassification, AutoProcessor
from transformers import ViTHybridForImageClassification, ViTHybridImageProcessor
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive


## Function: Model Evaluation
This function loads a pre-trained model and evaluates it on the test dataset.

In [ ]:
def evaluate_model(model_name, dataset, labels, batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu"):
    """Loads a model and evaluates it on the dataset in batches, displaying all failed predictions."""
    print(f"Evaluating {model_name}...")

    # Load model and processor
    if "hybrid" in model_name:
        model = ViTHybridForImageClassification.from_pretrained(model_name).to(device)
        processor = ViTHybridImageProcessor.from_pretrained(model_name)
    else:
        model = AutoModelForImageClassification.from_pretrained(model_name).to(device)
        processor = AutoProcessor.from_pretrained(model_name)

    y_true, y_pred = [], []
    failed_predictions = []

    start_time = time.time()

    for example in tqdm(dataset, desc=f"Testing {model_name}"):
        image, label = example["image"], example["label"]
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=-1).cpu().item()

        y_true.append(label)
        y_pred.append(pred_label)

        if pred_label != label:  # Store failed predictions
            failed_predictions.append((image, label, pred_label))

    elapsed_time = time.time() - start_time
    print(f"Model {model_name} evaluation completed in {elapsed_time:.2f} seconds.")
    print(f"Total failed predictions: {len(failed_predictions)}")


    return y_true, y_pred, failed_predictions, elapsed_time


## Function: Generate Report
This function generates and saves a classification report and confusion matrix.

In [ ]:
def generate_report(y_true, y_pred, labels, model_name, output_dir, failed_predictions, elapsed_time):
    """Generates and saves classification report and confusion matrix."""
    model_safe_name = model_name.split("/")[-1]
    model_safe_name = model_safe_name.split("_")[0] + model_safe_name.split("_")[-1]
    output_dir = os.path.join(output_dir, model_name.split("/")[-1])
    os.makedirs(output_dir, exist_ok=True)

    report = classification_report(y_true, y_pred, target_names=labels, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype("float") / cm.sum(axis=1, keepdims=True)

    report["evaluation_time_sec"] = elapsed_time

    # Save JSON report
    report_path = os.path.join(output_dir, f"report.json")
    with open(report_path, "w") as f:
        json.dump(report, f, indent=4)

    # Save Excel report
    report_df = pd.DataFrame(report).transpose()
    excel_path = os.path.join(output_dir, f"report.xlsx")

    with pd.ExcelWriter(excel_path) as writer:
        report_df.to_excel(writer, sheet_name="Classification Report")
        pd.DataFrame(cm, index=labels, columns=labels).to_excel(writer, sheet_name="Confusion Matrix")
        pd.DataFrame(cm_normalized, index=labels, columns=labels).to_excel(writer, sheet_name="Normalized Confusion Matrix")

    # Display all failed predictions
    if failed_predictions:
        num_failures = len(failed_predictions)
        cols = 4  # Set columns for visualization
        rows = (num_failures // cols) + (num_failures % cols > 0)

        fig, axes = plt.subplots(rows, cols, figsize=(cols * 10, rows * 10))
        axes = axes.flatten()  # Flatten for easy iteration

        for i, (image, true_label, pred_label) in enumerate(failed_predictions):
            ax = axes[i]
            ax.imshow(image)  # Assuming images are PIL images
            ax.set_title(f"True: {labels[true_label]}\nPred: {labels[pred_label]}")
            ax.axis("off")

        # Hide extra subplots if any
        for j in range(i + 1, len(axes)):
            axes[j].axis("off")

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "failed_predictions.png"))
        plt.close()

    # Save confusion matrix plot
    def save_cm_plot(matrix, title, filename, fmt="d"):
        plt.figure(figsize=(10, 8))
        sns.heatmap(matrix, annot=True, fmt=fmt, cmap="Blues", xticklabels=labels, yticklabels=labels)
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.title(title, pad=20)
        plt.xticks(rotation=30)
        plt.yticks(rotation=30)
        plt.savefig(os.path.join(output_dir, filename), bbox_inches="tight", pad_inches=0.3)
        plt.close()

    save_cm_plot(cm, f"{model_safe_name} Confusion Matrix", f"confusion_matrix.png")
    save_cm_plot(cm_normalized, f"{model_safe_name} Confusion Matrix", f"normalized_confusion_matrix.png", fmt=".2f")


In [ ]:
def save_dataset_info(dataset, output_dir):
    info_path = os.path.join(output_dir, "dataset_info.json")
    # Get class distribution
    labels = dataset.features["label"].names
    label_counts = {label: 0 for label in labels}

    for example in dataset:
        label_counts[labels[example["label"]]] += 1

    dataset_info = {
        "num_samples": len(dataset),
        "num_classes": len(labels),
        "class_distribution": label_counts,
    }

    # Save dataset info as JSON
    with open(info_path, "w") as f:
        json.dump(dataset_info, f, indent=4)

    # Save class distribution plot
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(label_counts.keys()), y=list(label_counts.values()), palette="viridis")
    plt.xticks(rotation=30)
    plt.xlabel("Classes")
    plt.ylabel("Count")
    plt.title("Class Distribution in Test Dataset")
    plt.savefig(os.path.join(output_dir, "class_distribution.png"), bbox_inches="tight", pad_inches=0.3)
    plt.close()

    print("✅ Dataset info saved.")

## Main Function
This function loads the dataset, evaluates models, and saves reports to Google Drive.

In [ ]:

def main():
    """Mounts Google Drive, loads dataset, evaluates models, and saves reports."""
    drive.mount("/content/drive")

    models_path = [
        "SodaXII/resnet-152_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/convnextv2-base-1k-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/vit-hybrid-base-bit-384_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/vit-base-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/swin-base-patch4-window7-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/deit-base-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/dinov2-base_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/mobilevit-small_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/efficientnet-b0_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/convnextv2-femto-1k-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/vit-tiny-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/swin-tiny-patch4-window7-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/deit-tiny-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/resnet-152_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/convnextv2-base-1k-224_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/vit-hybrid-base-bit-384_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/vit-base-patch16-224_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/swin-base-patch4-window7-224_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/deit-base-patch16-224_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/dinov2-base_rice-leaf-disease-augmented-v4_tl"
    ]


    dataset = load_dataset("cvmil/rice-leaf-disease-augmented-v4", split="test")
    labels = dataset.features["label"].names

    output_dir = "/content/drive/Shareddrives/CS198-Drones/[v4] Model Evaluation/"
    os.makedirs(output_dir, exist_ok=True)

    save_dataset_info(dataset, output_dir)

    for model_name in models_path:
        try:
            y_true, y_pred, failed_predictions, elapsed_time = evaluate_model(model_name, dataset, labels)
            generate_report(y_true, y_pred, labels, model_name, output_dir, failed_predictions, elapsed_time)
        except Exception as e:
            print(f"⚠️⚠️⚠️ Error processing {model_name}: {e}")

    print("✅ Evaluation completed. Reports saved to Google Drive.")

if __name__ == "__main__":
    main()


### Test

In [4]:
import torch
import time
import os
import pandas as pd
from collections import Counter
from tqdm import tqdm
from transformers import AutoModelForImageClassification, AutoProcessor, ViTHybridForImageClassification, ViTHybridImageProcessor
from datasets import load_dataset
from google.colab import drive
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def evaluate_model(model_name, dataset, labels, batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu"):
    """Loads a model and evaluates it on the dataset, returning predictions and accuracy."""
    print(f"Evaluating {model_name}...")

    # Load model and processor
    if "hybrid" in model_name:
        model = ViTHybridForImageClassification.from_pretrained(model_name).to(device)
        processor = ViTHybridImageProcessor.from_pretrained(model_name)
    else:
        model = AutoModelForImageClassification.from_pretrained(model_name).to(device)
        processor = AutoProcessor.from_pretrained(model_name)

    y_true, predictions = [], []

    for example in tqdm(dataset, desc=f"Testing {model_name}"):
        image, label = example["image"], example["label"]
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=-1).cpu().item()

        y_true.append(label)
        predictions.append(pred_label)

    accuracy = accuracy_score(y_true, predictions)
    return y_true, predictions, accuracy

def ensemble_prediction(models, dataset, labels, output_dir):
    """Runs multiple models, returns votes, computes accuracy, and saves reports."""
    all_predictions = {}
    model_accuracies = {}
    y_true = None

    for model_name in models:
        try:
            y_true, preds, acc = evaluate_model(model_name, dataset, labels)
            all_predictions[model_name] = preds
            model_accuracies[model_name] = acc
        except Exception as e:
            print(f"⚠️ Error processing {model_name}: {e}")

    # Transpose list of lists to get per-sample predictions
    per_sample_votes = list(zip(*all_predictions.values()))
    final_predictions = [Counter(votes).most_common(1)[0][0] for votes in per_sample_votes]
    ensemble_accuracy = accuracy_score(y_true, final_predictions)

    # Create a DataFrame with votes per sample, including True Labels
    df = pd.DataFrame(per_sample_votes, columns=models, index=[f"Sample_{i+1}" for i in range(len(per_sample_votes))])
    df.insert(0, "True Label", y_true)  # Add True Label as the first column
    df["Final Prediction"] = final_predictions
    df.to_csv(os.path.join(output_dir, "ensemble_votes.csv"))

    # Generate model accuracy report
    report_data = {"Model": list(model_accuracies.keys()) + ["Ensemble"],
                   "Accuracy": list(model_accuracies.values()) + [ensemble_accuracy]}
    report_df = pd.DataFrame(report_data)
    report_df.to_csv(os.path.join(output_dir, "model_accuracies.csv"), index=False)

    # Calculate classification report and confusion matrix for ensemble
    report = classification_report(y_true, final_predictions, target_names=labels, output_dict=True)
    cm = confusion_matrix(y_true, final_predictions)

    # Save classification report
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv(os.path.join(output_dir, "classification_report.csv"))

    # Save confusion matrix
    cm_df = pd.DataFrame(cm, index=labels, columns=labels)
    cm_df.to_csv(os.path.join(output_dir, "confusion_matrix.csv"))

    print(f"✅ Ensemble vote report saved to {output_dir}/ensemble_votes.csv")
    print(f"✅ Model accuracies saved to {output_dir}/model_accuracies.csv")
    print(f"✅ Classification report saved to {output_dir}/classification_report.csv")
    print(f"✅ Confusion matrix saved to {output_dir}/confusion_matrix.csv")

    return all_predictions

def main():
    """Mounts Google Drive, loads dataset, evaluates models with ensemble method, and saves results."""
    drive.mount("/content/drive")

    models_path = [
        "SodaXII/mobilevit-small_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/efficientnet-b0_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/convnextv2-femto-1k-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/vit-tiny-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/swin-tiny-patch4-window7-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/deit-tiny-patch16-224_rice-leaf-disease-augmented-v4_fft",
    ]

    dataset = load_dataset("cvmil/rice-leaf-disease-augmented-v4", split="test")
    labels = dataset.features["label"].names

    output_dir = "/content/drive/Shareddrives/CS198-Drones/Ensemble_Results"
    os.makedirs(output_dir, exist_ok=True)

    all_predictions = ensemble_prediction(models_path, dataset, labels, output_dir)
    print("✅ Ensemble evaluation completed.")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Evaluating SodaXII/mobilevit-small_rice-leaf-disease-augmented-v4_fft...


Testing SodaXII/mobilevit-small_rice-leaf-disease-augmented-v4_fft: 100%|██████████| 298/298 [00:11<00:00, 26.40it/s]


Evaluating SodaXII/efficientnet-b0_rice-leaf-disease-augmented-v4_fft...


Testing SodaXII/efficientnet-b0_rice-leaf-disease-augmented-v4_fft: 100%|██████████| 298/298 [00:07<00:00, 37.96it/s]


Evaluating SodaXII/convnextv2-femto-1k-224_rice-leaf-disease-augmented-v4_fft...


...

Evaluating SodaXII/swin-tiny-patch4-window7-224_rice-leaf-disease-augmented-v4_fft...


Testing SodaXII/swin-tiny-patch4-window7-224_rice-leaf-disease-augmented-v4_fft: 100%|██████████| 298/298 [00:14<00:00, 20.54it/s]


Evaluating SodaXII/deit-tiny-patch16-224_rice-leaf-disease-augmented-v4_fft...


Testing SodaXII/deit-tiny-patch16-224_rice-leaf-disease-augmented-v4_fft: 100%|██████████| 298/298 [00:10<00:00, 29.57it/s]

✅ Ensemble vote report saved to /content/drive/Shareddrives/CS198-Drones/Ensemble_Results/ensemble_votes.csv
✅ Model accuracies saved to /content/drive/Shareddrives/CS198-Drones/Ensemble_Results/model_accuracies.csv
✅ Classification report saved to /content/drive/Shareddrives/CS198-Drones/Ensemble_Results/classification_report.csv
✅ Confusion matrix saved to /content/drive/Shareddrives/CS198-Drones/Ensemble_Results/confusion_matrix.csv
✅ Ensemble evaluation completed.
